# Script to get true ranks for model evaluation

- Select/edit user query
- Run the code block below
- Choose and rank the 3 (out of the 6) most relevant dandisets
- Submit

In [37]:
# -----------------------------------------------------

user_query = "Are there any dandisets that include EEG data?" # Edit query here

# -----------------------------------------------------

import sys
sys.path.append("..")

import json
import dandi
from rest.clients.qdrant import QdrantClient
import ipywidgets as widgets
from IPython.display import display, clear_output


# Populate Qdrant
def populate_qdrant():
    with open("../data/qdrant_points.json", "r") as file:
        emb = json.load(file)

    qdrant_client = QdrantClient(host="https://906c3b3f-d3ff-4497-905f-2d7089487cf9.us-east4-0.gcp.cloud.qdrant.io")
    qdrant_client.create_collection(collection_name="dandi_collection")
    qdrant_client.add_points_to_collection(collection_name="dandi_collection", embeddings_objects=emb)

    return qdrant_client


# Get semantic search results
def get_similar_results(qdrant_client: QdrantClient):
    similar_results = qdrant_client.query_from_user_input(
        text=user_query,
        collection_name="dandi_collection",
        top_k=6, # adjust based on model's number of similar results
    )
    return similar_results


client = populate_qdrant()
similar_results = get_similar_results(client)


# Create interface for user feedback
options = [f"https://dandiarchive.org/dandiset/{dandiset_id[0].split(':')[-1]}" for dandiset_id in similar_results]

# display information about each relevant dandiset


initial_value = "Choose a dandiset"
rank_dd = [widgets.Dropdown(options=[initial_value] + options, description=f"Rank #{i+1}:", value=initial_value) for i in range(3)]

submit_button = widgets.Button(description="Submit Ranks")


def on_button_click(b):
    ranks = [rank.value for rank in rank_dd]
    if initial_value in ranks:
        print("Ensure each rank has a designated Dandiset!")
    elif len(set(ranks)) != len(ranks):
        print("Please avoid selecting the same Dandiset for multiple rankings!")

    else:            
        # store prompt and corresponding (rank: int, dandiset_id: str) pair

        # end script
        clear_output()
        print("Submission complete. Thank you!")


submit_button.on_click(on_button_click)


# Show display
display(rank_dd[0], rank_dd[1], rank_dd[2])
display(submit_button)

All points added to collection dandi_collection


Dropdown(description='Rank #1:', options=('Choose a dandiset', 'https://dandiarchive.org/dandiset/000574/draft…

Dropdown(description='Rank #2:', options=('Choose a dandiset', 'https://dandiarchive.org/dandiset/000574/draft…

Dropdown(description='Rank #3:', options=('Choose a dandiset', 'https://dandiarchive.org/dandiset/000574/draft…

Button(description='Submit Ranks', style=ButtonStyle())